<a href="https://colab.research.google.com/github/richiedinc/Maison/blob/master/MAISON_DNC_Stock_Prediction_Program(%5BDATA_YAHOO%5D).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This program predicts stock prices by using machine learning models

#Install the dependencies
import  fix_yahoo_finance as fyf
from pandas_datareader import data as pdr
import numpy as np 
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [ ]:
#Get the stock data
fyf.pdr_override()
df = pdr.get_data_yahoo("JPM", start ='2000-1-01', end='2020-07-1')
df.head
# Take a look at the data
print(df.head())

[*********************100%***********************]  1 of 1 downloaded
                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2000-01-03  49.833332  50.250000  48.083332  48.583332  26.551420  12019200
2000-01-04  47.083332  47.458332  46.125000  47.250000  25.968838  11723400
2000-01-05  46.833332  48.375000  46.000000  46.958332  25.808540   8714500
2000-01-06  46.750000  48.625000  46.500000  47.625000  26.174952   8369200
2000-01-07  48.416668  49.000000  47.333332  48.500000  26.655846   6571900


In [ ]:
# Get the Adjusted Close Price
df = df[['Close']]
#Take a look at the new data
print(df.head())

                Close
Date                 
2000-01-03  48.583332
2000-01-04  47.250000
2000-01-05  46.958332
2000-01-06  47.625000
2000-01-07  48.500000


In [ ]:
# A variable for predicting 'n' days out into the future
forecast_out = 1 #'n=30' days
#Create another column (the target or dependent variable) shifted 'n' units up
df['Prediction'] = df[['Close']].shift(-forecast_out)
#print the new data set
print(df.tail())

                Close  Prediction
Date                             
2020-06-24  94.660004   97.959999
2020-06-25  97.959999   92.589996
2020-06-26  92.589996   93.000000
2020-06-29  93.000000   94.059998
2020-06-30  94.059998         NaN


In [ ]:
### Create the independent data set (X)  #######
# Convert the dataframe to a numpy array
X = np.array(df.drop(['Prediction'],1))

#Remove the last 'n' rows
X = X[:-forecast_out]
print(X)

[[48.583332]
 [47.25    ]
 [46.958332]
 ...
 [97.959999]
 [92.589996]
 [93.      ]]


In [ ]:
### Create the dependent data set (y)  #####
# Convert the dataframe to a numpy array (All of the values including the NaN's)
y = np.array(df['Prediction'])
# Get all of the y values except the last 'n' rows
y = y[:-forecast_out]
print(y)

[47.25     46.958332 47.625    ... 92.589996 93.       94.059998]


In [ ]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.9973507174652251


In [ ]:
# Create and train the Linear Regression  Model
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.9975797181788465


In [ ]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[94.059998]]


In [ ]:
# Print linear regression model predictions for the next 'n' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

# Print support vector regressor model predictions for the next 'n' days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[94.05201194]
[93.76387761]


In [ ]:
model = lr.score(x_test, y_test)

In [ ]:
model.save('AMD_11_15.h5')

AttributeError: ignored